# Exploration des données des logs de distribution

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
# Lire un fichier Parquet
df = pd.read_parquet('../../data/logParquet/DetailedDistributionLogs_sub_2411222036.parquet')

# Afficher les premières lignes
print(df.head())

In [ ]:
# Drop colonnes inutiles

df = df.drop(columns=['FirstName', 'LastName', 'IsError']) 

In [ ]:
# Afficher les premières lignes
print(df.head())

In [ ]:
n_unique_login = df['Login'].nunique()
print(f"Nombre de login différents : {n_unique_login}")

In [ ]:
n_unique_distrib_name = df['DistributionName'].nunique()
print(f"Nombre de login différents : {n_unique_distrib_name}")

In [ ]:
from IPython.display import display

display(df.head())  # Permet de voir un tableau interactif

In [ ]:
# Compter les occurrences des Login
login_counts = df['Login'].value_counts()

# Sélectionner les 15 plus fréquents
top_login = login_counts.head(15)

plt.figure(figsize=(12, 6))
top_login.plot(kind='bar', color='skyblue', edgecolor='black')
plt.xticks(rotation=90)
plt.xlabel("Login")
plt.ylabel("Fréquence")
plt.title("Top 15 des Login les plus fréquents")
plt.show()

In [ ]:
# Compter les occurrences des erreur
error_counts = df['IsError'].value_counts()

plt.figure(figsize=(12, 6))
error_counts.plot(kind='bar', color='skyblue', edgecolor='black')
plt.xticks(rotation=90)
plt.xlabel("Login")
plt.ylabel("Fréquence")
plt.title("Fréquence des erreurs")
plt.show()

In [ ]:
# Compter les occurrences des ScheduleName
ScheduleName_counts = df['ScheduleName'].value_counts()

# Sélectionner les 15 plus fréquents
top_ScheduleName = ScheduleName_counts.head(15)

plt.figure(figsize=(12, 6))
top_ScheduleName.plot(kind='bar', color='skyblue', edgecolor='black')
plt.xticks(rotation=90)
plt.xlabel("Login")
plt.ylabel("Fréquence")
plt.title("Top 15 des Login les plus fréquents")
plt.show()

In [ ]:
# Compter les occurrences des DistributionName
DistributionName_counts = df['DistributionName'].value_counts()

# Sélectionner les 15 plus fréquents
top_DistributionName = DistributionName_counts.head(15)

plt.figure(figsize=(12, 6))
top_DistributionName.plot(kind='bar', color='skyblue', edgecolor='black')
plt.xticks(rotation=90)
plt.xlabel("Login")
plt.ylabel("Fréquence")
plt.title("Top 15 des Login les plus fréquents")
plt.show()

In [ ]:
# Sélectionner les 15 logins les plus fréquents
top_login = df['Login'].value_counts().head(15).index
df_top_login = df[df['Login'].isin(top_login)]

# Sélectionner les 15 dashboards les plus utilisés par ces logins
top_DistributionName = df_top_login['DistributionName'].value_counts().head(15).index
df_top = df_top_login[df_top_login['DistributionName'].isin(top_DistributionName)]

# Créer la matrice Login x DashboardName
login_DistributionName_matrix = df_top.pivot_table(index='Login', columns='DistributionName', aggfunc='size', fill_value=0)

# Affichage de la heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(login_DistributionName_matrix, annot=True, fmt="d", cmap="Blues", linewidths=0.5)
plt.xlabel("DistributionName")
plt.ylabel("Login")
plt.title("Connexion des Logins aux Distributions (Top 15 Logins & Distributions)")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Convertir en datetime
df['DatetimeLog'] = pd.to_datetime(df['DatetimeLog'])

In [ ]:
# Extraire le jour de la semaine en texte
jours_semaine = ["lundi", "mardi", "mercredi", "jeudi", "vendredi", "samedi", "dimanche"]
df['jour_semaine'] = df['DatetimeLog'].dt.weekday.map(lambda x: jours_semaine[x])

# Extraire l'heure arrondie à la tranche de 30 min
df['heure'] = df['DatetimeLog'].dt.hour + (df['DatetimeLog'].dt.minute // 60) * 0.5

# Formater l'affichage de l'heure en "9h00", "9h30", etc.
df['horaire'] = df['heure'].apply(lambda x: f"{int(x)}h{int((x % 1) * 60):02d}")

# Concaténer jour et horaire pour une meilleure lisibilité
df['jour_horaire'] = df['jour_semaine'].astype(str) + " " + df['horaire']

# Séparer jour et heure
df[['jour', 'horaire']] = df['jour_horaire'].str.split(' ', expand=True)

# Convertir le jour en catégorie ordonnée
df['jour'] = pd.Categorical(df['jour'], categories=jours_semaine, ordered=True)

# Convertir l'heure en entier pour tri correct
df['heure_num'] = df['horaire'].str.replace('h', '').astype(int)

# Trier d'abord par jour, puis par heure
df = df.sort_values(['jour', 'heure_num']).drop(columns=['jour', 'heure_num'])

In [ ]:
# Compter la fréquence des valeurs
freq_logs = df['jour_horaire'].value_counts().reset_index()
freq_logs.columns = ['jour_horaire', 'count']

# Fusionner avec le dataframe trié pour conserver l'ordre correct
freq_logs = df[['jour_horaire']].drop_duplicates().merge(freq_logs, on='jour_horaire', how='left')

# Taille du graphique
plt.figure(figsize=(12, 6))

# Tracer l'histogramme
plt.bar(freq_logs['jour_horaire'], freq_logs['count'], color='steelblue', edgecolor='black')

# Espacer les labels (afficher une étiquette toutes les 5 valeurs)
step = 2  # Modifier ce nombre si besoin
plt.xticks(range(0, len(freq_logs), step), freq_logs['jour_horaire'][::step], rotation=90, fontsize=10)

# Ajouter les labels
plt.xlabel("Jour et Horaire")
plt.ylabel("Nombre de logs")
plt.title("Fréquence des logs par jour et tranche horaire (étiquettes réduites)")

# Afficher le graphique
plt.show()

In [ ]:
# Sélectionner les 15 DistributionName les plus fréquents
top_dashboards = df['DistributionName'].value_counts().head(15).index
df_top = df[df['DistributionName'].isin(top_dashboards)]

# Compter les occurrences par DistributionName et jour_horaire
freq_logs = df_top.groupby(['DistributionName', 'jour_horaire']).size().reset_index(name='count')

# Trier les jours et horaires
freq_logs[['jour', 'horaire']] = freq_logs['jour_horaire'].str.split(' ', expand=True)
freq_logs['jour'] = pd.Categorical(freq_logs['jour'], categories=jours_semaine, ordered=True)
freq_logs = freq_logs.sort_values(['DistributionName', 'jour', 'horaire'])

# Générer un graphique par DistributionName
dashboard_list = freq_logs['DistributionName'].unique()
fig, axes = plt.subplots(len(dashboard_list), 1, figsize=(12, 5 * len(dashboard_list)))

if len(dashboard_list) == 1:
    axes = [axes]  # S'assurer que axes est une liste si un seul graphique

for ax, dashboard in zip(axes, dashboard_list):
    subset = freq_logs[freq_logs['DistributionName'] == dashboard]
    ax.bar(subset['jour_horaire'], subset['count'], color='steelblue', edgecolor='black')
    ax.set_title(f"Fréquence des logs pour {dashboard}")
    ax.set_xlabel("Jour et Horaire")
    ax.set_ylabel("Nombre de logs")
    ax.set_xticks(range(0, len(subset), max(1, len(subset) // 15)))
    ax.set_xticklabels(subset['jour_horaire'][::max(1, len(subset) // 15)], rotation=90, fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Sélectionner les 15 Login les plus fréquents
top_dashboards = df['Login'].value_counts().head(15).index
df_top = df[df['Login'].isin(top_dashboards)]

# Compter les occurrences par Login et jour_horaire
freq_logs = df_top.groupby(['Login', 'jour_horaire']).size().reset_index(name='count')

# Trier les jours et horaires
freq_logs[['jour', 'horaire']] = freq_logs['jour_horaire'].str.split(' ', expand=True)
freq_logs['jour'] = pd.Categorical(freq_logs['jour'], categories=jours_semaine, ordered=True)
freq_logs = freq_logs.sort_values(['Login', 'jour', 'horaire'])

# Générer un graphique par Login
dashboard_list = freq_logs['Login'].unique()
fig, axes = plt.subplots(len(dashboard_list), 1, figsize=(12, 5 * len(dashboard_list)))

if len(dashboard_list) == 1:
    axes = [axes]  # S'assurer que axes est une liste si un seul graphique

for ax, dashboard in zip(axes, dashboard_list):
    subset = freq_logs[freq_logs['Login'] == dashboard]
    ax.bar(subset['jour_horaire'], subset['count'], color='steelblue', edgecolor='black')
    ax.set_title(f"Fréquence des logs pour {dashboard}")
    ax.set_xlabel("Jour et Horaire")
    ax.set_ylabel("Nombre de logs")
    ax.set_xticks(range(0, len(subset), max(1, len(subset) // 15)))
    ax.set_xticklabels(subset['jour_horaire'][::max(1, len(subset) // 15)], rotation=90, fontsize=10)

plt.tight_layout()
plt.show()